In [11]:
from osgeo import gdal, ogr

In [4]:
import time
import os
import subprocess
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from typing import Union, Optional
from pydantic import BaseModel, validator

In [5]:
ROOT = 'assets'

class Config(BaseModel):
    input_file: Union[str, None] = 'Khon Kaen_F1_transparent_mosaic_group1.tif'
    output_mbtiles: Union[str, None] = None
    output_geojson: Union[str, None] = None
    
    @validator('output_mbtiles', pre=True)
    def output_file_join_path_mbtiles(cls, value):
        basename = os.path.basename(cls.input_file)
        output_file = os.path.join(ROOT, f'{basename}.mbtiles')
        return output_file
    
    @validator('output_geojson', pre=True)
    def output_file_join_path_geojson(cls, value):
        basename = os.path.basename(cls.input_file)
        output_file = os.path.join(ROOT, f'{basename}.geojson')
        return output_file

### Convert bytes 

- from os.path.getsize

In [6]:
def convert_bytes(size):
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if size < 1024.0:
            return "%3.1f %s" % (size, x)
        size /= 1024.0

In [7]:
def result_measure_area(tif: str) -> dict:
    ds = gdal.Open(tif)
    
    # Get the GeoTransform matrix (affine transformation coefficients)
    gt = ds.GetGeoTransform()

    # Compute the pixel size (assumes square pixels)
    pixel_size = gt[1]
    
    # Get the band object
    band = ds.GetRasterBand(1)
    
    # Compute statistics for the band
    stats = band.GetStatistics(True, True)
    
    # Get the minimum and maximum pixel values
    min_val = stats[0]
    max_val = stats[1]
    
    # Compute the number of pixels with non-zero values
    nz_pixels = (band.ReadAsArray() != 0).sum()
    
    # Compute the area in square meters
    area_m2 = nz_pixels * pixel_size * pixel_size
    
    # Convert area from square meters to rai
    rai = area_m2 / 1600
    res = {
        'min_pixel': min_val,
        'max_pixel': max_val,
        'number_non_zero_pixel': nz_pixels,
        'pixel_size': f'{pixel_size} m',
        'area (sq.m)': area_m2,
        'area (rai)': rai
    }
    return res


## Config Scheme
- path input
- path output_mbtiles
- path output_geojson (optional)

In [8]:
config = Config()
tifs = []
root_path = 'assets/nakhonsawan'
folder_tif = os.listdir(root_path)
for fn in folder_tif:
    if not fn.startswith('.'):
        if fn.endswith('.tif') or fn.endswith('.TIF'):
            file_tif = os.path.join(root_path, fn)
            tifs.append(file_tif)

tifs.sort()
tifs

['assets/nakhonsawan/20221102_NKSW_02_XAG_ndvi.tif',
 'assets/nakhonsawan/20221102_NKSW_03_XAG_ndvi.tif',
 'assets/nakhonsawan/20221102_NKSW_04_XAG_ndvi.tif',
 'assets/nakhonsawan/20221103_NKSW_05_XAG_ndvi.tif']

## Run Process convert .tif to .mbtiles

In [21]:
lst = []
count = 0

while count < 3:
    results = []
    for k, tif in enumerate(tifs):
        start_time = time.time()
        config.input_file = tif
        basename = os.path.basename(config.input_file).split('.')[0]
        config.output_geojson = f'output_geojson/{basename}.geojson'
        src_ds = gdal.Open(config.input_file)
    
        # Create output GeoJSON file
        out_driver = ogr.GetDriverByName('GeoJSON')
        out_ds = out_driver.CreateDataSource(config.output_geojson)
        
        # Create output layer
        out_layer = out_ds.CreateLayer('output', geom_type=ogr.wkbPolygon)
        
        # Add attribute fields to layer
        field_defn = ogr.FieldDefn('id', ogr.OFTInteger)
        out_layer.CreateField(field_defn)
        field_defn = ogr.FieldDefn('value', ogr.OFTReal)
        out_layer.CreateField(field_defn)
        
        # Polygonize the input raster
        gdal.Polygonize(src_ds.GetRasterBand(1), None, out_layer, 0, [], callback=None)
        
        # Close output file
        out_ds = None
        
        
        end_time = time.time() - start_time
        minute = end_time / 60
        sz = os.path.getsize(tif)
        obj = {'time_taken': round(minute, 4), 'file_tif': tif, 'output_geojson': config.output_geojson, 'file_size': convert_bytes(sz)}
        measure = result_measure_area(config.input_file)
        obj.update(measure)
        results.append(obj)
    lst.append(results)
    count += 1



# Measure

In [55]:
for n in range(len(lst)):
    df = pd.DataFrame(lst[n])
    df['time_taken'] = df['time_taken'] * 60
    df.to_csv(f'convert_geojson{n + 1}.csv')
    display(df)
    


,time_taken,file_tif,output_geojson,file_size,min_pixel,max_pixel,number_non_zero_pixel,pixel_size,area (sq.m),area (rai)
0,14.088,assets/nakhonsawan/20221102_NKSW_02_XAG_ndvi.tif,output_geojson/20221102_NKSW_02_XAG_ndvi.geojson,76.5 MB,-1.000000,1.000000,31185064,0.24198000000000003 m,1.826020e+06,1141.262643
1,15.384,assets/nakhonsawan/20221102_NKSW_03_XAG_ndvi.tif,output_geojson/20221102_NKSW_03_XAG_ndvi.geojson,63.5 MB,-1.000000,1.000000,23508028,0.24191000000000001 m,1.375700e+06,859.812708
2,22.434,assets/nakhonsawan/20221102_NKSW_04_XAG_ndvi.tif,output_geojson/20221102_NKSW_04_XAG_ndvi.geojson,82.7 MB,-1.000000,0.843052,33604997,0.24214000000000002 m,1.970321e+06,1231.450486
3,18.534,assets/nakhonsawan/20221103_NKSW_05_XAG_ndvi.tif,output_geojson/20221103_NKSW_05_XAG_ndvi.geojson,125.6 MB,-0.799532,0.960532,45687218,0.24179000000000003 m,2.670985e+06,1669.365376


,time_taken,file_tif,output_geojson,file_size,min_pixel,max_pixel,number_non_zero_pixel,pixel_size,area (sq.m),area (rai)
0,13.518,assets/nakhonsawan/20221102_NKSW_02_XAG_ndvi.tif,output_geojson/20221102_NKSW_02_XAG_ndvi.geojson,76.5 MB,-1.000000,1.000000,31185064,0.24198000000000003 m,1.826020e+06,1141.262643
1,13.842,assets/nakhonsawan/20221102_NKSW_03_XAG_ndvi.tif,output_geojson/20221102_NKSW_03_XAG_ndvi.geojson,63.5 MB,-1.000000,1.000000,23508028,0.24191000000000001 m,1.375700e+06,859.812708
2,26.916,assets/nakhonsawan/20221102_NKSW_04_XAG_ndvi.tif,output_geojson/20221102_NKSW_04_XAG_ndvi.geojson,82.7 MB,-1.000000,0.843052,33604997,0.24214000000000002 m,1.970321e+06,1231.450486
3,18.630,assets/nakhonsawan/20221103_NKSW_05_XAG_ndvi.tif,output_geojson/20221103_NKSW_05_XAG_ndvi.geojson,125.6 MB,-0.799532,0.960532,45687218,0.24179000000000003 m,2.670985e+06,1669.365376


,time_taken,file_tif,output_geojson,file_size,min_pixel,max_pixel,number_non_zero_pixel,pixel_size,area (sq.m),area (rai)
0,13.356,assets/nakhonsawan/20221102_NKSW_02_XAG_ndvi.tif,output_geojson/20221102_NKSW_02_XAG_ndvi.geojson,76.5 MB,-1.000000,1.000000,31185064,0.24198000000000003 m,1.826020e+06,1141.262643
1,13.500,assets/nakhonsawan/20221102_NKSW_03_XAG_ndvi.tif,output_geojson/20221102_NKSW_03_XAG_ndvi.geojson,63.5 MB,-1.000000,1.000000,23508028,0.24191000000000001 m,1.375700e+06,859.812708
2,21.822,assets/nakhonsawan/20221102_NKSW_04_XAG_ndvi.tif,output_geojson/20221102_NKSW_04_XAG_ndvi.geojson,82.7 MB,-1.000000,0.843052,33604997,0.24214000000000002 m,1.970321e+06,1231.450486
3,21.042,assets/nakhonsawan/20221103_NKSW_05_XAG_ndvi.tif,output_geojson/20221103_NKSW_05_XAG_ndvi.geojson,125.6 MB,-0.799532,0.960532,45687218,0.24179000000000003 m,2.670985e+06,1669.365376
